# 역전파 (Backpropagation)
- "AI가 자기가 틀린 걸 알고, 어디서 틀렸는지를 계산해서 스스로 고치는 방법"
- 즉, 오차를 거꾸로 전달하면서 가중치를 수정하는 과정이야.
---
- 💡 딥러닝에서는 이렇게 적용돼:
- 모델이 입력 x를 보고 예측값 ŷ를 냄
- 실제 정답 y와 비교해서 loss(손실) 계산
- 이 loss가 왜 발생했는지를 각 층별로 거꾸로 추적
- 각 가중치 W, 편향 b가 얼마나 영향을 줬는지 계산 → 기울기(gradient)
- 그 기울기만큼 가중치를 조정해서 더 잘 맞추도록 함
- 이 전 과정을 "역전파"라고 해! (→ loss.backward())
---
- 💡 한줄 요약
- 역전파(Backpropagation) 는
- 딥러닝에서 오차가 발생했을 때,
- 그 오차를 각 층으로 거슬러 올라가며 계산하고,
- 각 가중치를 얼마나 수정해야 하는지를 알려주는 방법이야.

### 연쇄법칙

- 기본 수식의 역전파 & 연쇄법칙 적용

In [1]:
import numpy as np

def forward(x):
    y = x**2
    return y

def backward(x):
    dy_dx = 2 * x
    return dy_dx

x = 3.0
print(forward(x))
print(backward(x))

9.0
6.0


- 다층 신경망에서 연쇄법칙 적용

In [2]:
def forward(x):
    y = x**2
    z = 2 * y
    return z

def backward(x):
    dy_dx = 2 * x
    dz_dy = 2
    dz_dx = dz_dy * dy_dx
    return dz_dx

x = 3.0
print(forward(x))
print(backward(x))

18.0
12.0


### 신경망에서의 활용

- 단순 신경망 학습

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_d(x):
    return sigmoid(x) * (1 - sigmoid(x))

X = np.array([0.5, 0.8])
y = np.array([1])

W = np.array([0.2, 0.4])

# 순전파
z = np.dot(X, W)
r = sigmoid(z)

# 오차 계산
loss = 0.5 * (y - r) ** 2

# 역전파 (기울기 계산)
delta = (r - y) * sigmoid_d(z)
grad_w = delta * X

# 가중치 갱신
W -= 0.1 * grad_w   # 0.1 == learning_rate

print(W)

[0.20474415 0.40759064]


- 은닉층 추가

In [5]:
def relu(x):
    return np.maximum(0, x)

def relu_d(x):
    return np.where(x > 0, 1, 0)

X = np.array([0.5, 0.8])   # (2,)
y = np.array([1])

W1 = np.array([[0.2, 0.4], [0.1, 0.3]])   # (2, 2)
b1 = np.array([0.1, 0.2])   # (2,)
W2 = np.array([[0.5], [0.6]])   # (2, 1)
b2 = np.array([0.3])

# 순전파
z1 = np.dot(X, W1) + b1
r1 = relu(z1)

z2 = np.dot(r1, W2) + b2
r2 = relu(z2)

# 역전파 (기울기 계산)
delta2 = (r2 - y) * relu_d(z2)
grad_W2 = np.outer(r1, delta2)

delta1 = np.dot(W2, delta2) * relu_d(z1)
grad_W1 = np.outer(X, delta1)

# 가중치 갱신
learning_rate = 0.01
W2 -= learning_rate * grad_W2
W1 -= learning_rate * grad_W1

print(W2)
print(W1)

[[0.5004928]
 [0.6011264]]
[[0.20044   0.400528 ]
 [0.100704  0.3008448]]


### 수치미분과 역전파

In [6]:
def f(x):
    return x**2

def num_d_gradient(f, x):
    h = 1e-5
    return (f(x + h) - f(x - h)) / (2 * h)

def backward_gradient(x):
    return 2 * x

print(num_d_gradient(f, 3.0))
print(backward_gradient(3.0))

6.000000000039306
6.0


##### 숫자 맞추기 AI

In [10]:
target_number = 42      # 우리가 맞추고 싶은 "정답"

guess = np.random.randn()   # 랜덤하게 시작하는 예측값 (정답을 모르니까 대충 추측)

learning_rate = 0.1     # 한 번에 얼마만큼 이동할지 (기울기를 얼마나 반영할지)

for i in range(500):
    # 오차 계산
    loss = 0.5 * (guess - target_number) ** 2
        # 손실(Loss) = 예측값과 정답 사이의 차이(오차)를 계산
        # 수식: MSE (Mean Squared Error) 형식
        # 0.5는 미분 시 깔끔하게 사라지도록 관습적으로 붙임

    # 역전파 (기울기 계산)
    grad = (guess - target_number)
    # 📌 역전파 단계 (기울기 계산)
    # 오차를 줄이기 위해 어느 방향으로 움직일지 결정
    # 여기서는 loss를 guess에 대해 미분한 값이야

    # 업데이트 (guess 업데이트)
    guess -= learning_rate * grad
    # 🔥 경사하강법(Gradient Descent)
    # 기울기 방향으로 guess를 조금 수정!
    # learning_rate가 너무 크면 튕기고, 작으면 너무 느림

    # epoch 5마다 예측값과 손실 출력
    if i % 50 == 0:
        print(f'epoch {i} | 예측값: {guess} , 손실: {loss}')
        # 📢 50번마다 중간 결과 출력
        # → guess가 점점 42에 가까워지고, loss는 줄어드는 걸 볼 수 있어!



# 최종 예측값 guess 출력
print(f'최종 예측값: {guess}')

epoch 0 | 예측값: 3.51776248838994 , 손실: 914.1250641357832
epoch 50 | 예측값: 41.80167119839046 , 손실: 0.024280440461652383
epoch 100 | 예측값: 41.998977857939366 , 손실: 6.44922464272799e-07
epoch 150 | 예측값: 41.999994732109585 , 손실: 1.713004283434827e-11
epoch 200 | 예측값: 41.999999972850475 , 손실: 4.549980490536778e-16
epoch 250 | 예측값: 41.99999999986007 , 손실: 1.2086059951568363e-20
epoch 300 | 예측값: 41.999999999999275 , 손실: 3.223348767605472e-25
epoch 350 | 예측값: 41.99999999999997 , 손실: 4.0389678347315804e-28
epoch 400 | 예측값: 41.99999999999997 , 손실: 4.0389678347315804e-28
epoch 450 | 예측값: 41.99999999999997 , 손실: 4.0389678347315804e-28
최종 예측값: 41.99999999999997
